In [2]:
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 7.4 합성곱/풀링 계층 구현하기

## 7.4.1 4차원 배열

In [ ]:
# 높이 28, 너비 28, 채널 1, 데이터 10개 
x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

In [ ]:
x[0].shape
x[1].shape

In [ ]:
# 첫 번째 데이터의 첫 채널의 공간 데이터에 접근하려면..
x[0, 0] # 또는 x[0][0]
x[0, 0].shape

## 7.4.3 합성곱 계층 구현하기

In [ ]:
im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [4]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [5]:
class Convolution:
    # W : Weight => 가중치, 필터
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int((H + 2*self.pad - FH)/self.stride + 1)
        out_w = int((W + 2*self.pad - FW)/self.stride + 1)

        # 입력 데이터를 im2col로 전개
        col = im2col(x, FH, FW, self.stride, self.pad)
        
        # 필터도 reshape을 사용해 2차원 배열로 전개
        col_W = self.W.reshape(FN, -1).T

        # 이렇게 전개한 두 행렬의 곱을 구함
        out = np.dot(col, col_W) + self.b

        # 출력 데이터를 적절한 형상으로 바꿔줌
        # (N, C, OH, OW)
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

## 7.4.4 풀링 계층 구현하기

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int((H - self.pool_h)/self.stride + 1)
        out_w = int((W - self.pool_w)/self.stride + 1)

        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # 최댓값 (2)
        out = np.max(col, axis=1)

        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

# 7.5 CNN 구현하기

In [11]:
class SimpleConvNet:

    # 초기화를 수행하는 메서드
    def __init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, ouput_size=10, weight_init_std=0.01):
                
                # 초기화 인수로 주어진 합성곱 계층의 하이퍼파라미터를 딕셔너리에서 꺼냄
                filter_num = conv_param['filter_num']
                filter_size = conv_param['filter_size']
                filter_pad = conv_param['pad']
                filter_stride = conv_param['stride']
                input_size = input_dim[1]

                # 합성곱 계층의 출력 크기를 계산
                conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_stride + 1
                pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
                
                # 가중치 매개변수를 초기화 
                self.params = {}
                # W1, b1 : 1번째 층의 합성곱 계층의 가중치와 편향
                self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
                self.params['b1'] = np.zeros(filter_num)
                # W2, b2 : 2번째 층의 완전연결계층의 가중치와 편향 
                self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
                self.params['b2'] = np.zeros(hidden_size)
                # W3, b3 : 마지막 3번째 층의 완전연결계층의 가중치와 편향 
                self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
                self.params['b3'] = np.zeros(output_size)

                # 순서가 있는 딕셔너리인 layers에 계층들을 차례로 추가
                self.layers = OrderedDict()
                self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
                self.layers['Relu1'] = Relu()
                self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
                self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
                self.layers['Relu2'] = Relu()
                self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
                
                # SoftmaxWithLoss 계층만큼은 last_layer라는 별도 변수에 저장 
                self.last_layer = SoftmaxWithLoss()

    # 추론을 수행하는 predict 메서드
    def predict(self, x):
        # 초기화 때 layers에 추가한 계층을 맨 앞에서부터 차례로 forward 메서드를 호출하며 그 결과를 다음 계층에 전달함
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    # 손실 함수의 값을 구하는 loss 메서드
    # x : 입력 데이터, t : 정답 레이블
    # 손실 함수를 구하는 loss 메서드는 predict 메서드의 결과를 인수로 마지막 층의 forward 메서드를 호출함
    # 즉, 첫 계층부터 마지막 계층까지 forward를 처리함
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads

In [13]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

ModuleNotFoundError: No module named 'dataset.mnist'